# Transformation to C Code (Medium)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np

from library.fvm.solver import Settings
from library.model.models.shallow_water import ShallowWaterEquations

import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
from library.misc.misc import Zstruct
import library.transformation.to_c as trafo

2025-08-19 07:43:03.667 | WARNING  | library.fvm.solver:<module>:28 - No module named 'precice'


## Model definition

In [2]:
bcs = BC.BoundaryConditions(
    [
        BC.Wall(physical_tag="top"),
        BC.Wall(physical_tag="bottom"),
        BC.Wall(physical_tag="left"),
        BC.Wall(physical_tag="right"),
    ]
)

def custom_ic(x):
    Q = np.zeros(3, dtype=float)
    Q[0] = np.where(x[0] < 5., 0.005, 0.001)
    return Q

ic = IC.UserFunction(custom_ic)

model = ShallowWaterEquations(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)



## Code transformation

In [3]:
settings = Settings(name="ShallowWater", output=Zstruct(directory="outputs/trafo", filename="swe.h5"))

trafo.to_c(model, settings)

2025-08-19 07:43:04.160 | WARNING  | library.misc.misc:__init__:146 - No 'clean_directory' attribute found in output Zstruct. Default: False


## Check the output

In [4]:
main_dir = os.getenv("ZOOMY_DIR")
path = os.path.join(main_dir, os.path.join(settings.output.directory, '.c_interface/Model/model_code.c'))
with open(path, "r") as f:
    print(f.read())


/******************************************************************************
 *                      Code generated with SymPy 1.13.3                      *
 *                                                                            *
 *              See http://www.sympy.org/ for more information.               *
 *                                                                            *
 *                       This file is part of 'project'                       *
 ******************************************************************************/
#include "model_code.h"
#include <math.h>

void flux_x(double *Q, double *Qaux, double *parameters, double *out_4207687440490456541) {

   out_4207687440490456541[0] = Q[1];
   out_4207687440490456541[1] = (1.0/2.0)*pow(Q[0], 2)*parameters[0] + pow(Q[1], 2)/Q[0];
   out_4207687440490456541[2] = Q[1]*Q[2]/Q[0];

}

void flux_y(double *Q, double *Qaux, double *parameters, double *out_4848542861252137306) {

   out_4848542861252137306[0] 